In [1]:
# Importação de bibliotecas
import pandas as pd

from pendulum import now

In [6]:
# Objetos
fonte = 'sidra'
dataset_name = 'tab5457_fixbioN'
variavel = 'quant_produzida'

raiz = '/home/eduardo/repositorios/IS_Agro/databases'

### Bronze

Quantidade produzida (t)

In [3]:
contador = 1

ultimo_ano = now().year - 1
for ano in range(2000, ultimo_ano):
    if contador == 1:
        df_5457 = pd.read_json(f'https://apisidra.ibge.gov.br/values/t/5457/n6/all/v/214/p/{ano}/c782/40101,40106,40110,40112,40124')
        contador += 1
    else:
        df_5457 = pd.concat(
            [df_5457,
            (pd.read_json(f'https://apisidra.ibge.gov.br/values/t/5457/n6/all/v/214/p/{ano}/c782/40101,40106,40110,40112,40124')
             .iloc[1:])],
            ignore_index=True            
        )
        contador += 1

df_5457.to_json(f'{raiz}/bronze_{fonte}_{dataset_name}_{variavel}.json')

### Prata

In [9]:
(pd.read_json(f'{raiz}/bronze_{fonte}_{dataset_name}_{variavel}.json')
 .drop(0, axis = 0)
 .filter(['D1C', 'D3C', 'D4C', 'V'])
 .assign(**{
        'uf': lambda df: df['D1C'].str.slice(0,2).astype(int),
        'safra_t': lambda df: pd.to_numeric(df['V'], errors='coerce'),
        'municipio': lambda df: df['D1C'].astype(int),
        'data': lambda df: pd.to_datetime(df['D3C'], format='%Y'), # Coluna "data" em datetime baseado na ISO 8601, convertido da coluna "D3C"
        'cultura': lambda df: df['D4C'].astype(int)
        })
 .dropna(subset='safra_t')
 .convert_dtypes()
 .groupby(['uf', 'municipio', 'data', 'cultura'])
 .agg({'safra_t':'sum'})
 .to_parquet(f'{raiz}/prata_{fonte}_{dataset_name}_{variavel}.parquet')
 )

/tmp/ipykernel_56595/3130921525.py:14: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  .agg({'safra_t':sum})


| CÓDIGO | CULTURA AGRÍCOLA |
|--------|------------------|
| 40112  |      Feijão      |
| 40106  |  Cana-de-açúcar  |
| 40124  |       Soja       |
| 40101  |     Amendoim     |
| 40110  |     Ervilha      |


### Ouro

Parâmetros para cálculos
| Fonte               | Teor | Unidade             | Fonte              | Teor | Unidade             |
|---------------------|------|---------------------|--------------------|------|---------------------|
| N-planta            | 2    | kg N/kg N produto   |
| N-planta Soja       | 80   | kg N/t de soja      | Pfix%-Soja         | 80   | % do N planta       |
| N-Amendoim          | 59,5 | kg N/t de produto   | Pfix%-Amendoim     | 65   | % do N planta       |
| N-Ervilha           | 36   | kg N/t de produto   | Pfix%-Ervilha      | 65   | % do N planta       |
| N-Cana-de-açúcar    | 0,8  | kg N/t de produto   | Pfix%-Cana-de-açúcar | 35  | % do N planta      |
| N-Feijão            | 38,5 | kg N/t de produto   | Pfix%-Feijão        | 36   | % do N planta      |

Específico para soja -> (Safra_ton(soja) * N-planta soja * (Pfix%-soja/100))/1000

Para outras culturas -> (Safra_ton * N-planta * N-cultura{código} * (Pfix%-cultura/100))/1000

In [11]:
n_planta = 2 # fator geral

parametro = {
    40124: {'n': 80, 'pfix': 0.80}, # soja
    40101: {'n': 59.5, 'pfix': 0.65}, # amendoim
    40106: {'n': 0.8, 'pfix': 0.35}, # cana-de-açúcar
    40110: {'n': 36, 'pfix': 0.65}, # ervilha
    40112: {'n': 38.5, 'pfix': 0.36} # feijão
    }

In [14]:
df = (pd.read_parquet(f'{raiz}/prata_{fonte}_{dataset_name}_{variavel}.parquet')
      .groupby(['uf', 'data', 'cultura'])
      .agg({'safra_t':'sum'})
      .reset_index()
      .assign(
          fonte = 'Fixação biológica de N',
          elemento = 'N',
          valor=lambda x: x.apply
          (lambda linha: (linha['safra_t'] * parametro[linha['cultura']]['n'] * parametro[linha['cultura']]['pfix'])/1000
           if linha['cultura'] == 40124 else 
           (linha['safra_t'] * n_planta * parametro[linha['cultura']]['n'] * parametro[linha['cultura']]['pfix'])/1000, 
           axis = 1))
      .drop('safra_t', axis = 1)
      .convert_dtypes()
      .groupby(['uf', 'data', 'fonte'])
      .agg({'elemento': 'max', 'valor':'sum'})
      )

elemento        valor
uf data       fonte                                       
11 2000-01-01 Fixação biológica de N        N     3710.361
   2001-01-01 Fixação biológica de N        N   5593.38814
   2002-01-01 Fixação biológica de N        N   6358.92556
   2003-01-01 Fixação biológica de N        N   9055.11055
   2004-01-01 Fixação biológica de N        N  11626.97751
...                                       ...          ...
53 2018-01-01 Fixação biológica de N        N   17214.8324
   2019-01-01 Fixação biológica de N        N  17658.50348
   2020-01-01 Fixação biológica de N        N  19906.59848
   2021-01-01 Fixação biológica de N        N   20745.9548
   2022-01-01 Fixação biológica de N        N  20417.55952

[621 rows x 2 columns]

In [16]:
df = (
    df
    .rename_axis(index={
        'uf':'state',
        'data': 'date',
        'fonte': 'label'
    })
    .rename(columns={
        'elemento':'nutrient',
        'valor':'value'
    })
)

df.to_csv(f'{raiz}/ouro_npk_fixbioN.csv')